In [ ]:
%load_ext ipybind
%matplotlib inline

import warnings
warnings.filterwarnings('default')

import cxxfilt
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
plt.rc('figure', figsize=(8, 8))

# C++ array library: xtensor

Yung-Yu Chen

PyHUG June 2019

## What is xtensor

* https://xtensor.readthedocs.io/
* “xtensor is a C++ library meant for numerical analysis with multi-dimensional array expressions.”
* Why?  Help to deal with multi-dimensional array, which is hard.

# Multi-dimensional arrays in Python

numpy N-dimensional array (ndarry) makes life very easy.

In [ ]:
arr = np.zeros((2,3), dtype='float64')
print(arr)

In [ ]:
print(type(arr))

For example, see how easy it is to transpose:

In [ ]:
mat = np.arange(9, dtype='float64').reshape((3,3))
print(mat)

In [ ]:
print(mat.T)

# What's wrong with Python and numpy?

It's usually slow when the problem becomes complex.

This will be demonstrated using a boundary value problem with the Laplace equation for temperature distribution in a $1\times1$ square area.

\begin{align}
& \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = 0 \quad (0<x<1; 0<y<1) \\
& u(0,y) = 0, \, u(1,y) = \sin(\pi y) \quad (0<y<1) \\
& u(x,0) = u(x,1) = 0 \quad (0<x<1)
\end{align}

Make the grid.

In [ ]:
def make_grid():
    nx = 51
    x = np.linspace(0, 1, nx)
    gx, gy = np.meshgrid(x, x)
    u = np.zeros_like(gx)
    u[0,:] = 0
    u[-1,:] = 1 * np.sin(np.linspace(0,np.pi,nx))
    u[:,0] = 0
    u[:,-1] = 0
    return nx, x, u

In [ ]:
def show_grid(size):
    fig, ax = plt.subplots(figsize=(size,size))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks(x, minor=True)
    ax.set_yticks(x, minor=True)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.grid(True, which='minor')

In [ ]:
nx, x, uoriginal = make_grid(); show_grid(10)

Use the Taylor series expansion to obtain the difference equation:

\begin{align}
& \frac{u(x_{i+1}, y_j) - 2u(x_i, y_j) + u(x_{i-1}, y_j)}{(\Delta x)^2} \\
&\quad + \frac{u(x_i, y_{j+1}) - 2u(x_i, y_j) + u(x_i, y_{j+1})}{(\Delta y)^2} = 0
\end{align}

Now we can use the point-Jacobi method to write a formula to iteratively solve the difference equaion:

\begin{align}
u^{n+1}(x_i, y_i) = \frac{u^n(x_{i+1}, y_j) + u^n(x_{i-1}, y_j) + u^n(x_i, y_{j+1}) + u^n(x_i, y_{j-1})}{4}
\end{align}

The most straight-forward solver implementation in Python is a nested loop:

In [ ]:
def solve_python_loop():
    u = uoriginal.copy()
    un = u.copy()
    converged = False
    step = 0
    while not converged:
        step += 1
        for it in range(1, nx-1):
            for jt in range(1, nx-1):
                un[it,jt] = (u[it+1,jt] + u[it-1,jt] + u[it,jt+1] + u[it,jt-1]) / 4
        norm = np.abs(un-u).max()
        u[...] = un[...]
        converged = True if norm < 1.e-5 else False
    return u, step, norm

In [ ]:
def show_result(u, step, norm, size=7):
    print("step", step, "norm", norm)
    fig, ax = plt.subplots(figsize=(size,size))
    cs = ax.contour(x, x, u.T)
    ax.clabel(cs, inline=1, fontsize=10)

    ax.set_xticks(np.linspace(0,1,6))
    ax.set_yticks(np.linspace(0,1,6))
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')

    ax.grid(True, which='minor')

In [ ]:
%%time
u, step, norm = solve_python_loop()

In [ ]:
show_result(u, step, norm)

In [ ]:
ugolden = u.copy()

# Array-based code with numpy

There is a cure for the slow runtime: use array to delegate the calculation in C.

In [ ]:
def solve_array():
    u = uoriginal.copy()
    un = u.copy()
    converged = False
    step = 0
    while not converged:
        step += 1
        un[1:nx-1,1:nx-1] = (u[2:nx,1:nx-1] + u[0:nx-2,1:nx-1] +
                             u[1:nx-1,2:nx] + u[1:nx-1,0:nx-2]) / 4
        norm = np.abs(un-u).max()
        u[...] = un[...]
        converged = True if norm < 1.e-5 else False
    return u, step, norm

In [ ]:
%%time
u, step, norm = solve_array()

In [ ]:
assert (u == ugolden).all(); show_result(u, step, norm)

# Nested loop in C++

Is that fast enough?  No, because we haven't seen the speed of C++.

```cpp
const size_t nx = u.shape(0);
xt::xarray<double> un = u;
bool converged = false;
size_t step = 0;
double norm;
while (!converged) {
    ++step;
    for (size_t it=1; it<nx-1; ++it) {
        for (size_t jt=1; jt<nx-1; ++jt) {
            un(it,jt) = (u(it+1,jt) + u(it-1,jt) +
                         u(it,jt+1) + u(it,jt-1)) / 4;
        }
    }
    norm = xt::amax(xt::abs(un-u))();
    if (norm < 1.e-5) { converged = true; }
    u = un;
}
```

And it's not the convoluted array code.  The nested loop is closer to the iterative formula.

In [ ]:
%%pybind11 -c="-O3"

#include "pybind11/pybind11.h"
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"

#include <vector>
#include <algorithm>
#include <tuple>
#include <iostream>

#include "xtensor/xarray.hpp"
#include "xtensor/xadapt.hpp"
#include "xtensor/xview.hpp"

std::tuple<xt::xarray<double>, size_t, double>
solve1(xt::xarray<double> u)
{
    const size_t nx = u.shape(0);
    xt::xarray<double> un = u;
    bool converged = false;
    size_t step = 0;
    double norm;
    while (!converged)
    {
        ++step;
        for (size_t it=1; it<nx-1; ++it)
        {
            for (size_t jt=1; jt<nx-1; ++jt)
            {
                un(it,jt) = (u(it+1,jt) + u(it-1,jt) + u(it,jt+1) + u(it,jt-1)) / 4;
            }
        }
        norm = xt::amax(xt::abs(un-u))();
        if (norm < 1.e-5) { converged = true; }
        u = un;
    }
    return std::make_tuple(u, step, norm);
}

PYBIND11_MODULE(code1, m)
{
    xt::import_numpy();
    m.def
    (
        "solve_cpp", [](xt::pyarray<double> & uin) { return solve1(xt::xarray<double>(uin)); }
    );
}

In [ ]:
%%time
u, step, norm = solve_cpp(uoriginal)

In [ ]:
assert (u == ugolden).all(); show_result(u, step, norm)

# Array-based code in C++

We can also write array code with xtensor:

```cpp
const size_t nx = u.shape(0);
xt::xarray<double> un = u;
auto uxlower = xt::view(u, xt::range(0, nx-2), xt::range(1, nx-1));
auto uxupper = xt::view(u, xt::range(2, nx), xt::range(1, nx-1));
auto uylower = xt::view(u, xt::range(1, nx-1), xt::range(0, nx-2));
auto uyupper = xt::view(u, xt::range(1, nx-1), xt::range(2, nx));
auto uncenter = xt::view(un, xt::range(1, nx-1), xt::range(1, nx-1));
bool converged = false;
size_t step = 0;
double norm;
while (!converged) {
    ++step;
    uncenter = (uxupper + uxlower + uyupper + uylower) / 4;
    norm = xt::amax(xt::abs(un-u))();
    if (norm < 1.e-5) { converged = true; }
    u = un;
}
```

In [ ]:
%%pybind11 -c="-O3"

#include "pybind11/pybind11.h"
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"

#include <vector>
#include <algorithm>
#include <tuple>
#include <iostream>

#include "xtensor/xarray.hpp"
#include "xtensor/xadapt.hpp"
#include "xtensor/xview.hpp"

std::tuple<xt::xarray<double>, size_t, double>
solve2(xt::xarray<double> u)
{
    const size_t nx = u.shape(0);
    xt::xarray<double> un = u;
    auto uxlower = xt::view(u, xt::range(0, nx-2), xt::range(1, nx-1));
    auto uxupper = xt::view(u, xt::range(2, nx), xt::range(1, nx-1));
    auto uylower = xt::view(u, xt::range(1, nx-1), xt::range(0, nx-2));
    auto uyupper = xt::view(u, xt::range(1, nx-1), xt::range(2, nx));
    auto uncenter = xt::view(un, xt::range(1, nx-1), xt::range(1, nx-1));
    bool converged = false;
    size_t step = 0;
    double norm;
    while (!converged)
    {
        ++step;
        uncenter = (uxupper + uxlower + uyupper + uylower) / 4;
        norm = xt::amax(xt::abs(un-u))();
        if (norm < 1.e-5) { converged = true; }
        u = un;
    }
    return std::make_tuple(u, step, norm);
}

PYBIND11_MODULE(code2, m)
{
    xt::import_numpy();
    m.def
    (
        "solve_cpp_array", [](xt::pyarray<double> & uin) { return solve2(xt::xarray<double>(uin)); }
    );
}

In [ ]:
%%time
u, step, norm = solve_cpp_array(uoriginal)

In [ ]:
assert (u == ugolden).all(); show_result(u, step, norm)

References:
* xtensor; multi-dimensional arrays with broadcasting and lazy computing: https://xtensor.readthedocs.io
* xtensor-python; Python bindings for the xtensor C++ multi-dimensional array library: https://xtensor-python.readthedocs.io
* pybind11 — Seamless operability between C++11 and Python: https://pybind11.readthedocs.io/en/stable/
* IPython / Jupyter integration for pybind11: https://github.com/aldanor/ipybind